In [ ]:
file_name = "WhatsApp Chat with السلنتح.txt"

In [ ]:
#هنا ناخذ الرسالة ونحولها من  الشكل الاول للشكل الثاني
#03/11/2023, 22:30 - ْبلوخي حكايم: خليك البرمجه تنفعك
#['22:30', 'ْبلوخي حكايم', 'خليك البرمجه تنفعك']
#ناخذ المفيد منها لبرنامجنا ونحذف الباقي
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


def extract_content(file_name):
    with open(file_name, "r", encoding="utf-8") as conversation: #يفتح الملف اللي اخترناه
        messages = conversation.readlines() #يحوله لقائمة لكل سطر
        data = []
        for message in messages: #يقرا كل سطر
            match = re.search(r'^\d{2}/\d{2}/\d{4}, (\d{2}:\d{2}) - ([^:]+): (.+)$', message) #نمط الريجيكس حق الرسالة, قروشة شوي شفلها مقطع شرح ذي
            if match:
                time = match.group(1) 
                name = match.group(2)
                content = match.group(3)
                if content != '<Media omitted>' and content != "This message was deleted":
                    content = content.replace("This message was edited", "").strip()
                    data.append([time, name, content])
    return data


In [ ]:
#تعطيه الرسايل مفلتره ويحسبلك كم رسالة من كل شخص
def count_messages(names):
    message_counts = {}
    for name in names:
        if name in message_counts:
            message_counts[name] += 1
        else:
            message_counts[name] = 1
    sorted_message_counts = dict(sorted(message_counts.items(), key=lambda item: item[1], reverse=True)) #يرتبهن من الاكثر للاقل
    return sorted_message_counts

# print(count_messages(data))

In [ ]:
#يحسب تكرارات الرسايل بكل المحادثة
def get_common_words(contents, top_x=10):
    words = {}
    for content in contents:
        for word in content.split():
            if len(word) >= 5 and word.isalnum():  #نحسب الكلمات اللي تتجاوز 4 حروف بس, عشان نحذف حروف العطف والجر
                if word in words: #اذا كانت الكلمة موجوده يزيد تكراراتها 1
                    words[word] += 1
                else: #اذا مهب موجوده يحسبلها اول تكرار
                    words[word] = 1
    top_words = sorted(words.items(), key=lambda x: x[1], reverse=True)[:top_x] #نرتبها حسب التكرار
    return top_words

In [ ]:
#هنا يرسملنا كثافة الرسايل بالاوقات ومنهو المرسل, ذا نقشته من ابو جيبيتي
def plot_message_density(times, names):
    hours = [datetime.strptime(time, '%H:%M').hour + datetime.strptime(time, '%H:%M').minute / 60 for time in times]
    data = {'hours': hours, 'names': names}
    
    plt.figure(figsize=(10, 5))
    sns.kdeplot(data=data, x='hours', hue='names', bw_adjust=0.5, fill=True, alpha=0.5)
    plt.xlabel("Time")
    plt.ylabel('Messages')
    plt.title('Conversation Time')
    plt.xticks(range(0, 25, 1))
    plt.xlim(0, 24)
    plt.yticks([]) 
    plt.show()


In [ ]:
data = extract_content(file_name)

times = [entry[0] for entry in data] #نعطيه محتوى الرسايل بس
names = [entry[1] for entry in data] #محتوى الاسماء بس
contents = [entry[2] for entry in data] #نعطيه محتوى الرسايل بس

print("- عدد رسايلكم -")
message_counts = count_messages(names)
for member,count in message_counts.items():
    print(f'{member}: {count}')


top_length = 10 #كم نبي التوب كلمات
print("\n- أكثر كلمات كررتوها -")
top_words = get_common_words(contents, top_length)
for word,frequency in top_words:
    print(f'{word}: {frequency}')


print("\n- اوقات رسايلكم -")
plot_message_density(times,names)